In [8]:
import networkx as nx

with open("./semi.map", 'r') as ofn:
    lines = ofn.readlines()
    
for ind, l in enumerate(lines):
    sp = l.split()
    if len(sp) < 1:
        continue
    if len(sp) > 1:
        if sp[1] == "equivalences":
            numAtom = int(sp[0])
        if sp[1] == "edgeIDs":
            numEdge = int(sp[0])
            
    if sp[0] == "InitiatorIDs":        
        InitIDs = list(map(int,lines[ind+2:ind+2+2]))
    
    if sp[0] == "EdgeIDs":        
        EdgeIDs  = list(map(int,lines[ind+2:ind+2+numEdge]))
    
    if sp[0] == "Equivalences":
        pre_post_map_bef  = {int(l.split()[0]):int(l.split()[1]) for l in lines[ind+2:ind+2+numAtom] }

In [9]:
class Molecules:
    def __init__(self, ifn):
        with open(ifn, "r") as ofn:
            lines = ofn.readlines()
            
        for ind, l in enumerate(lines):
            sp = l.split()
            if len(sp) < 1:
                continue
            if len(sp) > 1:
                if sp[1] == "atoms":
                    self.numAtom = int(sp[0])
                if sp[1] == "bonds":
                    self.bonds = int(sp[0])
                if sp[1] == "angles":
                    self.angles = int(sp[0])
                if sp[1] == "dihedrals":
                    self.dihedrals = int(sp[0])
                if sp[1] == "impropers":
                    self.impropers = int(sp[0])
                    
            if sp[0] == "Types":        
                self.InitIDs = [ [int(l.split()[0]),l.split()[1]] for l in lines[ind+2:ind+2+self.numAtom] ]

            if sp[0] == "Charges":        
                self.Charges = {int(l.split()[0]):l.split()[1] for l in lines[ind+2:ind+2+self.numAtom] }

            if sp[0] == "Coords":        
                self.Coords = {int(l.split()[0]):"\t".join(l.split()[1:]) for l in lines[ind+2:ind+2+self.numAtom] }
            
            if sp[0] == "Bonds":
                self.Bonds = [ list(map(int, l.split()[1:])) for l in lines[ind+2:ind+2+self.bonds]]
                
            if sp[0] == "Angles":
                self.Angles = [ list(map(int, l.split()[1:])) for l in lines[ind+2:ind+2+self.angles]]
                
            if sp[0] == "Dihedrals":
                self.Dihedrals = [ list(map(int, l.split()[1:])) for l in lines[ind+2:ind+2+self.dihedrals]]         
                
            if sp[0] == "Impropers":
                self.Impropers = [ list(map(int, l.split()[1:])) for l in lines[ind+2:ind+2+self.impropers]]        
    
    def Delete_atoms(self, delete_atoms):
        
        self.InitIDs = {key:val for key,val in self.InitIDs if key not in delete_atoms}
        self.old2newid = {key:ind+1 for ind,key in enumerate(self.InitIDs)}
        self.new2old = {value: key for key, value in self.old2newid.items()}
        
        self.InitIDs = {self.old2newid[key]:val for key,val in self.InitIDs.items()}
        self.Charges = {self.old2newid[key]:val for key,val in self.Charges.items() if key in self.old2newid}
        self.Coords = {self.old2newid[key]:val for key,val in self.Coords.items() if key in self.old2newid}
        
        self.Bonds = [ [l[0]] + [self.old2newid[a] for a in l[1:]] for l in self.Bonds if all(elem in self.old2newid for elem in l[1:])]
        self.Angles = [ [l[0]] + [self.old2newid[a] for a in l[1:]] for l in self.Angles if all(elem in self.old2newid for elem in l[1:])]
        self.Dihedrals = [ [l[0]] + [self.old2newid[a] for a in l[1:]] for l in self.Dihedrals if all(elem in self.old2newid for elem in l[1:])]
        self.Impropers = [ [l[0]] + [self.old2newid[a] for a in l[1:]] for l in self.Impropers if all(elem in self.old2newid for elem in l[1:])]
        
        self.numAtom = len(self.InitIDs)
        self.bonds = len(self.Bonds)
        self.angles = len(self.Angles)
        self.dihedrals = len(self.Dihedrals)
        self.impropers = len(self.Impropers)
        
    def Write_molecule(self, ifn):
        with open(ifn, "w") as ofn:
            ofn.write("delete edged atoms \n")
            ofn.write(f"{self.numAtom} atoms\n{self.bonds} bonds\n{self.angles} angles\n{self.dihedrals} dihedrals\n{self.impropers} impropers\n")
            ofn.write("\nTypes\n\n")
            ofn.write("\n".join([ "\t".join([str(k),val]) for k,val in self.InitIDs.items()]))
            ofn.write("\n\nCharges\n\n")
            ofn.write("\n".join([ "\t".join([str(k),val]) for k,val in self.Charges.items()]))
            ofn.write("\n\nCoords\n\n")
            ofn.write("\n".join([ "\t".join([str(k),val]) for k,val in self.Coords.items()]))
            ofn.write("\n\nBonds\n\n")
            ofn.write("\n".join([ "\t".join([str(ind+1)] + list(map(str, val))) for ind,val in enumerate(self.Bonds)]))
            ofn.write("\n\nAngles\n\n")
            ofn.write("\n".join([ "\t".join([str(ind+1)] + list(map(str, val))) for ind,val in enumerate(self.Angles)]))
            ofn.write("\n\nDihedrals\n\n")
            ofn.write("\n".join([ "\t".join([str(ind+1)] + list(map(str, val))) for ind,val in enumerate(self.Dihedrals)]))
            ofn.write("\n\nImpropers\n\n")
            ofn.write("\n".join([ "\t".join([str(ind+1)] + list(map(str, val))) for ind,val in enumerate(self.Impropers)]))
            
    def Write_xyz(self,ifn):
        with open(ifn, "w") as ofn:
            ofn.write(f"{self.numAtom}\n")
            ofn.write("mol\n")
            ofn.write("\n".join([ "\t".join([str(v2),v1]) for v1,v2 in zip(self.Coords.values(),self.InitIDs.values())]))

In [10]:
pre_mol = Molecules("./pre-mol.data")

G = nx.Graph()
G.add_nodes_from([i for i in range(1,pre_mol.numAtom+1)])
G.add_edges_from([[b[1], b[2]] for b in pre_mol.Bonds])

In [11]:
G_del_edge = G.copy()

for edge in EdgeIDs:
    G_del_edge.remove_node(edge)

delete_atoms = []
for component in nx.connected_components(G_del_edge):
    if set(InitIDs).intersection(set(component)):
        pass
    else:
        delete_atoms.extend(component)

In [12]:
pre_mol.Delete_atoms(delete_atoms)
pre_mol.Write_molecule("cleaned_pre-mol")
#

In [13]:
post_mol = Molecules("./post-mol.data")
post_mol.Delete_atoms([pre_post_map_bef[v] for v in delete_atoms])
post_mol.Write_molecule("cleaned_post-mol")


In [14]:
with open("./cleaned_semi.map", 'w') as ofn:
    ofn.write("this map is created by cleaned-SemiMapper\n\n")
    ofn.write(f"{pre_mol.numAtom} equivalences\n{len(EdgeIDs)} edgeIDs\n\n")
    ofn.write("InitiatorIDs\n\n")
    ofn.write("\n".join([str(pre_mol.old2newid[id]) for id in InitIDs]))
    ofn.write("\n\nEdgeIDs\n\n")
    ofn.write("\n".join([str(pre_mol.old2newid[id]) for id in EdgeIDs]))
    ofn.write("\n\nEquivalences\n\n")
    ofn.write("\n".join( ["\t".join([str(ind), str(post_mol.old2newid[pre_post_map_bef[pre_mol.new2old[ind]]])]) for ind in range(1, pre_mol.numAtom+1)] ) )
    ofn.write("\n")
